In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from lgsm.paths import paths

In [ ]:
# Set some dimensions
ysize = 2.5
xsize = 6.25

ymin = -1
ymax = 3.2
xmin = -0.5
xmax = (ymax - ymin) * xsize / ysize + xmin

fig, ax = plt.subplots(figsize=(xsize, ysize), dpi=200, constrained_layout=True)
ax.set(
    aspect="equal",
    ylim=(ymin, ymax),
    xlim=(xmin, xmax),
)

# The first latent variable
x = np.linspace(0, 1, 100)
y = np.exp(-(x-0.5)**2 / (2 * 0.13**2))
y *= 0.85 / y.max()
ax.plot(x, y, color="silver")
ax.plot([0, 1, 1, 0, 0], [0, 0, 1, 1, 0], c="#262626", lw=0.5)
ax.text(0.5, 1.17, "$u_0$", ha="center", va="center", color="gray", size=8)

# Redshift flow
ax.text(1.9, 0.65, "Redshift\nflow", color="tomato", ha="center", size=7)
ax.arrow(1.2, 0.5, 1.4, 0, color="tomato", head_width=0.1, length_includes_head=True)
ax.plot([1.9, 1.9], [-0.25, 0.5], c="tomato", ls="--")
ax.text(1.9, -0.5, "$\mathbf{p}$", ha="center", va="center", size=8)
ax.text(2.8, 0.5, "$z$", ha="center", va="center", size=8)

# Other latent variables
x = np.linspace(1.4, 2.4, 1000)
y = np.linspace(1.5, 2.5, 1000)
x, y = np.meshgrid(x, y)
z = np.exp(-((x - 1.9)**2 + (y - 2)**2) / (2 * 0.23**2))
z *= 0.8 / z.max()
ax.contourf(x, y, z, levels=100, cmap="Grays", vmin=0, vmax=1)
ax.plot([1.4, 2.4, 2.4, 1.4, 1.4], [1.5, 1.5, 2.5, 2.5, 1.5], c="#262626", lw=0.5)
ax.text(1.9, 2.67, "$u_{1\!:\!11}$", ha="center", va="center", color="gray", size=8)

# Latent flow
ax.text(3.3, 2.15, "Latent\nflow", color="cornflowerblue", ha="center", size=7)
ax.arrow(2.6, 2, 1.4, 0, color="cornflowerblue", head_width=0.1, length_includes_head=True)
ax.plot([3.3, 3.3], [1.25, 2], c="cornflowerblue", ls="--")
ax.plot([2.8, 2.8, 3.8, 3.8, 2.15], [0.75, 1.25, 1.25, -0.5, -0.5], c="cornflowerblue", ls="--")
ax.text(4.2, 2, "$\mathbf{\\theta}$", ha="center", va="center", size=8)

# SED emulator
ax.text(5.2, 2.15, "SED\nemulator", color="C2", ha="center", size=7)
ax.arrow(4.45, 2, 1.5, 0, color="C2", head_width=0.1, length_includes_head=True)
ax.plot([6.1, 7.6, 7.6, 6.1, 6.1], [1.5, 1.5, 2.5, 2.5, 1.5], c="#262626", lw=0.5)
_, y = np.genfromtxt("data/Ell2_template_norm.sed.dat", unpack=True)
y = 0.85 * y / y.max() + 1.5
x = np.linspace(6.1, 7.6, y.size)
ax.plot(x, y, color="silver", lw=0.5)
ax.text(6.85, 2.67, "Latent SED", ha="center", va="center", color="gray", size=7)

# Physics layer
ax.text(8.5, 2.15, "Physics\nlayer", color="C4", ha="center", size=7)
ax.arrow(7.8, 2, 1.4, 0, color="C4", head_width=0.1, length_includes_head=True)
ax.plot([3.05, 8.5, 8.5], [0.5, 0.5, 2], c="C4", ls="--")
ax.text(9.4, 2, "$\hat{\mathbf{p}}$", ha="center", va="center", size=8)

ax.set_axis_off()
fig.savefig(paths.figures / "architecture.pdf")

Explanation:

1. The redshift flow predicts a redshift $z$ given the photometry $\mathbf{p}$. Note this step is skipped by spectroscopic data.
2. The latent flow predicts a set of latent galaxy SED parameters $\mathbf{\theta}$ given the photometry $\mathbf{p}$ and the redshift $z$.
3. The SED emulator predicts a rest-frame SED given the latent galaxy SED parameters $\mathbf{\theta}$.
4. The Physics layer predicts galaxy photometry $\mathbf{\hat{p}}$ given the SED and the redshift $z$. The physics layer includes (i) redshifting the rest-frame SED, (ii) Lyman-transition IGM extinction, (iii) calculating synthetic photometry using the photometric bandpasses, and (iv) zeropoint offsets for each photometric band.

You can then compare the input photometry $\mathbf{p}$ with the autoencoded photometry $\mathbf{\hat{p}}$.

Once everything is trained:
- The Redshift flow is a photo-z predictor that has been trained to be consistent with a latent SED model
- The Redshift flow + Latent flow + SED emulator is a probabilistic SED predictor